In [ ]:
import re
import subprocess
import base64
import pandas as pd
from smolagents import tool, ToolCallingAgent, CodeAgent, PromptTemplates, OpenAIServerModel #TransformersModel, HfApiModel
from jaws.jaws_config import *
from jaws.jaws_utils import dbms_connection

In [2]:
driver = dbms_connection(DATABASE)

In [3]:
# Help functions

def sanitize(output: str) -> str:
    cleaned = re.sub(r'\x1b\[[0-9;]*[a-zA-Z]', '', output)
    cleaned = re.sub(r'[╭╮╯╰─│]', '', cleaned)
    cleaned = re.sub(r'\s+', '\n', cleaned)
    lines = [line.strip() for line in cleaned.split('\n') if line.strip()]
    return '\n'.join(lines)


def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
EVIDANCE = f"{FINDER_ENDPOINT}/pca_dbscan_outliers.png"
base64_image = encode_image(EVIDANCE)

In [4]:
# Tools

@tool
def list_interfaces() -> str:
    """
    Lists available network interfaces.

    Returns:
        str: A string listing all available network interfaces.
    """ 
    interfaces = subprocess.run(['python', './jaws/jaws_capture.py', '--list'], capture_output=True, text=True)
    return sanitize(interfaces.stdout)


@tool
def capture_packets(interface: str, duration: int) -> str:
    """
    Captures packets into the database. 
    Choose a duration depending on the amount of data you want to capture.

    Args:
        interface (str): The network interface to capture packets from.
        duration (int): The duration of the capture in seconds.

    Returns:
        str: The output of the jaws_capture command.
    """
    packets = subprocess.run(['python', './jaws/jaws_capture.py', '--interface', interface, '--duration', str(duration)], capture_output=True, text=True)
    return sanitize(packets.stdout)


@tool
def document_organizations() -> str:
    """
    Documents organizations by sending IP addresses to ipinfo.io.

    Returns:
        str: The output of the jaws_ipinfo command.
    """
    organizations = subprocess.run(['python', './jaws/jaws_ipinfo.py'], capture_output=True, text=True)
    return sanitize(organizations.stdout)


@tool
def compute_embeddings() -> str:
    """
    Turns network data into embeddings for reporting and analysis.

    Returns:
        str: The output of the jaws_compute command.
    """
    embeddings = subprocess.run(['python', './jaws/jaws_compute.py'], capture_output=True, text=True)
    return sanitize(embeddings.stdout)


@tool
def fetch_data_for_advisor() -> pd.DataFrame:
    """
    Fetches the latest data from the database and returns it as a DataFrame.

    Returns:
        tuple: A tuple containing the DataFrame.
    """
    query = """
    MATCH (traffic:TRAFFIC)
    RETURN DISTINCT
        traffic.IP_ADDRESS AS ip_address,
        traffic.PORT AS port,
        traffic.ORGANIZATION AS org,
        traffic.HOSTNAME AS hostname,
        traffic.LOCATION AS location,
        traffic.TOTAL_SIZE AS total_size,
        traffic.OUTLIER AS outlier
    """
    with driver.session(database=DATABASE) as session:
        result = session.run(query)
        data = []
        for record in result:
            data.append({
                'ip_address': record['ip_address'],
                'port': record['port'],
                'org': record['org'],
                'hostname': record['hostname'],
                'location': record['location'],
                'total_size': record['total_size'],
                'outlier': record['outlier']
            })
        return pd.DataFrame(data)

In [5]:
network_analyst = ToolCallingAgent(
    name="network_analyst",
    description="Collect network traffic and OSINT.",
    model=OpenAIServerModel(model_id=OPENAI_MODEL),
    tools=[list_interfaces, capture_packets, document_organizations, compute_embeddings],
    verbosity_level=0
)

expert_analysis = CodeAgent(
    name="network_security_advisor",
    description="Reviews network traffic and provides recommendations for firewall configurations.",
    model=OpenAIServerModel(model_id=OPENAI_MODEL),
    planning_interval=1,
    max_steps=4,
    tools=[fetch_data_for_advisor],
    managed_agents=[network_analyst]
)

In [ ]:
#collect_packets = network_analyst.run(ANALYST_PROMPT)
analysis = expert_analysis.run(ADVISOR_PROMPT)